In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from gmplot import gmplot
from sklearn import metrics
import matplotlib.pyplot as plt
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn.cluster import DBSCAN
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')
# os.chdir("../")
# os.listdir()

In [13]:
city_name = 'delhi-1'

In [15]:
file_name = 'data/' + city_name + '-reduced_amenity.csv'
data = pd.read_csv(file_name)
data = data[['lat', 'lon']]
print(data.shape)
data.head()

(3499, 2)


,lat,lon
0,28.533492,77.151895
1,28.533647,77.152001
2,28.489238,77.090988
3,28.450926,77.087609
4,28.440015,77.102391


In [16]:
coords = data.values

In [17]:
kms_per_radian = 6371.0088
epsilon = 1 / kms_per_radian

dbscan = DBSCAN(eps = epsilon ,
                min_samples = 2.2 ,
                algorithm = 'ball_tree' ,
                metric = 'haversine')

model = dbscan.fit(np.radians(coords))


In [18]:
cluster_labels = dbscan.labels_
num_clusters = len(set(cluster_labels))
num_clusters

44

In [19]:
clusters = pd.Series([ coords [ cluster_labels == n ] for n in range (num_clusters) ])
clusters[0], len(clusters[0])

(array([[28.533492 , 77.1518947],
        [28.5336472, 77.152001 ],
        [28.5084497, 77.166546 ],
        ...,
        [28.509824 , 77.201705 ],
        [28.5100591, 77.201592 ],
        [28.6406954, 77.2113562]]), 2536)

In [20]:
# clusters = clusters[1:]

In [21]:
# clusters[101]

In [22]:
core_samples = np.zeros_like(cluster_labels, dtype='bool')
core_samples[dbscan.core_sample_indices_] = True
core_samples

array([ True,  True,  True, ...,  True,  True,  True])

In [23]:
len(core_samples), len(data)

(3499, 3499)

In [24]:
np.unique(core_samples, return_counts=True)

(array([False,  True]), array([ 102, 3397]))

In [25]:
s = pd.Series(core_samples, name='bools')

In [26]:
data[s.values].shape, data.shape

((3397, 2), (3499, 2))

In [28]:
file_name = "data/" + city_name + '-dbscan.csv'
data[s.values].to_csv(file_name, index=False)

#  Identifying a cluster’s centermost point

In [17]:
# def get_centermost_point(cluster):
#     centroid = (MultiPoint(cluster).centroid.x , MultiPoint(cluster).centroid.y)
#     centermost_point = min (cluster, key = lambda point : great_circle(point, centroid).m)
#     return tuple(centermost_point)
# centermost_points = clusters.map(get_centermost_point)
# centermost_points[:5]

In [ ]:
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon': lons ,'lat': lats })

In [154]:
df = data.copy()
rs = rep_points.apply(lambda row : df[(df['lat'] == row ['lat']) & (df['lon']== row['lon'])].iloc[0], axis =1)

rs

,lat,lon
0,28.587223,77.212080
1,28.495030,77.093564
2,28.462298,77.087271
3,28.497417,77.147735
4,28.489985,77.138963
5,28.612870,77.323059
6,28.526225,77.100783
7,28.473364,77.124522
8,28.432627,77.072277
9,28.651735,77.298207


# create polygons

In [155]:
len(clusters)

32

In [156]:
clusters[:5]

0    [[28.533492, 77.1518947], [28.5336472, 77.1520...
1    [[28.489238399999998, 77.09098840000001], [28....
2    [[28.4509257, 77.0876091], [28.4400152, 77.102...
3    [[28.604361199999996, 77.32309559999999], [28....
4    [[28.5262252, 77.1007834], [28.5229363, 77.099...
dtype: object

In [157]:
# Convex Hull
# point class with x, y as point 
class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 

def Left_index(points): 
    ''' Finding the left most point '''
    minn = 0
    for i in range(1,len(points)): 
        if points[i].x < points[minn].x: 
            minn = i 
        elif points[i].x == points[minn].x: 
            if points[i].y > points[minn].y: 
                minn = i 
    return minn 

def orientation(p, q, r): 
    ''' 
    To find orientation of ordered triplet (p, q, r). 
    The function returns following values 
    0 --> p, q and r are colinear 
    1 --> Clockwise 
    2 --> Counterclockwise 
    '''
    val = (q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y) 

    if val == 0: 
        return 0
    elif val > 0: 
        return 1
    else: 
        return 2

def convexHull(points, n): 
    # There must be at least 3 points 
    if n < 3: 
        return

    # Find the leftmost point 
    l = Left_index(points) 

    hull = [] 
    
    ''' 
    Start from leftmost point, keep moving counterclockwise 
    until reach the start point again. This loop runs O(h) 
    times where h is number of points in result or output. 
    '''
    p = l 
    q = 0
    while(True): 
        # Add current point to result 
        hull.append(p) 

        ''' 
        Search for a point 'q' such that orientation(p, x, 
        q) is counterclockwise for all points 'x'. The idea 
        is to keep track of last visited most counterclock- 
        wise point in q. If any point 'i' is more counterclock- 
        wise than q, then update q. 
        '''
        q = (p + 1) % n 
        
        for i in range(n): 
            
            # If i is more counterclockwise than current q, then update q 
            if(orientation(points[p], points[i], points[q]) == 2): 
                q = i 
        
        ''' 
        Now q is the most counterclockwise with respect to p 
        Set p as q for next iteration, so that q is added to 
        result 'hull' 
        '''
        p = q 

        # While we don't come to first point 
        if(p == l): 
            break

    # Print Result 
    result = []
    for each in hull: 
        result.append([points[each].x, points[each].y]) 
    return result

In [158]:
def apply_convex_hull(coordinates):
    points = []
    for coordinate in coordinates:
        x, y = coordinate[0], coordinate[1]
        points.append(Point(x,y))
        
    return convexHull(points, len(points))
    

In [159]:
polygon = []
for i in range(len(clusters)):
    if len(clusters[i]) < 10: continue
    polygon.append(apply_convex_hull(clusters[i]))

In [160]:
len(polygon[0]), len(clusters[0])

(13, 2534)

In [161]:
polygon = polygon[1:]
clusters = clusters[1:]

In [162]:
poly_coord = []
for i in range(len(polygon)):
#     print(i)
    if polygon[i] is None: continue
    lat = [x[0] for x in polygon[i]]
    long = [x[1] for x in polygon[i]]
    poly_coord.append([lat, long])
    
poly_coord[0]

[[28.489238399999998,
  28.493767300000002,
  28.497633299999997,
  28.5045702,
  28.5044048,
  28.492398100000003],
 [77.09098840000001,
  77.08592320000001,
  77.0865213,
  77.0971354,
  77.0973074,
  77.107998]]

In [163]:
min_lat, max_lat, min_lon, max_lon = min(data['lat']), max(data['lat']), min(data['lon']), max(data['lon'])
## Create empty map with zoom level 16
mymap = gmplot.GoogleMapPlotter(
    min_lat + (max_lat - min_lat) / 2, 
    min_lon + (max_lon - min_lon) / 2, 
    16)

In [164]:
for i in range(data.shape[0]):
    mymap.scatter([data.iloc[i]['lat']], [data.iloc[i]['lon']], 'red', size=20, marker=False)

In [165]:
for i in range(len(poly_coord)):
    
    lat = poly_coord[i][0]
    long = poly_coord[i][1]
    colour = np.random.rand(3,)

    mymap.scatter( lat, long, '# FF0000', size = 40, marker = False)

    mymap.polygon(lat, long, '# FF0000')

In [166]:
file_name = city_name  + '-poly-DBSCAN' "-" + str(len(clusters)) + '.html'
mymap.draw(file_name)
print(file_name)

delhi-1-polygon-DBSCAN-31.html
